# 1. Import Library

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import re
from tqdm import tqdm
from collections import defaultdict

# 2. Load Data

In [2]:
path = '/opt/ml'
data_path = path +'/data'
books_path = data_path + '/books.csv'
users_path = data_path + '/users.csv'
train_ratings_path = data_path + '/train_ratings.csv'
test_ratings_path = data_path + '/test_ratings.csv'

In [3]:
df_books = pd.read_csv(books_path) # book table
df_users = pd.read_csv(users_path) # users table
df_train_ratings = pd.read_csv(train_ratings_path) # train rating
df_test_ratings = pd.read_csv(test_ratings_path) # test_rating

In [4]:
def str_lower(x):
    try:
        return x.lower()
    except:
        return x

## 1) Users

In [5]:
for column in df_users.columns:
    if column == 'user_id':
        continue
    if df_users[column].dtype == object:
        df_users[column] = df_users[column].apply(str_lower)

### 1-1) Location 결측치 처리

In [6]:
df_users['location'] = df_users['location'].apply(lambda x : re.sub(r'[^a-zA-Z,]', '', x)) # 알파벳을 제외한 특수문자 제거
df_users['location'] = df_users['location'].apply(lambda x: ','.join(map(str, ['na' if '' == i else i for i in x.split(',')]))) # 공백은 na로 표시

df_users['location_city'] = df_users['location'].apply(lambda x: x.split(',')[0].strip()) # 도시
df_users['location_state'] = df_users['location'].apply(lambda x: x.split(',')[1].strip()) # 주
df_users['location_country'] = df_users['location'].apply(lambda x: x.split(',')[2].strip()) # 국가

df_users = df_users.replace('na', np.nan) # na를 NaN값으로 변환
df_users = df_users.replace('', np.nan)

In [7]:
# state 결측치는 city,country 정보, country 결측치는 city,state 정보 이용해서 해결
def fillna_location(to_fill:int,to_use:int):
    column_name = {0:'location_city',1:'location_state',2:'location_country'}
    to_fill_colname,to_use_colname = column_name[to_fill],column_name[to_use]
    
    modify_location = set(df_users[(df_users[to_fill_colname].isna()) & ((df_users[to_use_colname].notnull()))][to_use_colname].values)
    location_list = []
    for location in tqdm(modify_location): 
        try:
            candidates = df_users[(df_users[to_use_colname]==location) & ((df_users[to_use_colname].notnull()))]['location'].value_counts()
            first = candidates.idxmax()
            for k, v in dict(candidates).items():
                k = k.split(',')
                if 'na' not in k:
                    right_location = ','.join(map(str, k))
                    break
            else:
                right_location = first
            location_list.append(right_location)
        except:
            pass

    for location in tqdm(location_list): # [지역, 주, 국가]
        df_users.loc[df_users[(df_users[to_use_colname]==location.split(',')[to_use]) & (df_users[to_fill_colname].isna())].index, to_fill_colname] = location.split(',')[to_fill]

fillna_location(1,0) # state의 결측치를 city의 정보를 활용해 채워보자
fillna_location(1,2) # state의 결측치를 country의 정보를 활용해 채워보자
fillna_location(2,0) # country의 결측치를 city의 정보를 활용해 채워보자
fillna_location(2,1) # country의 결측치를 state의 정보를 활용해 채워보자


# 남은 결측치는 unknown으로 채우기
df_users[['location_city', 'location_state', 'location_country']] = df_users[['location_city', 'location_state', 'location_country']].fillna('unknown')

# where가 들어간 city, state, country는 다 unknown으로 교체
df_users.loc[df_users[df_users['location_city'].str.contains('where')].index, 'location_city'] = 'unknown'
df_users.loc[df_users[df_users['location_state'].str.contains('where')].index, 'location_state'] = 'unknown'
df_users.loc[df_users[df_users['location_country'].str.contains('where')].index, 'location_country'] = 'unknown'

100%|██████████| 2/2 [00:00<00:00, 113.09it/s]


In [8]:
temp = df_users.groupby('location_city')['location_country'].value_counts().groupby('location_city').idxmax().apply(lambda x : x[-1]).reset_index().rename(columns = {'count':'location_country'})
city2country = dict(zip(temp['location_city'].values, temp['location_country'].values))

df_users['location_country'] = df_users['location_city'].map(city2country) #location city에 대해 Location_country가 max인 나라로 채워서 noise 해결

In [9]:
df_users = df_users.drop('location', axis=1)

### 1-2) age 결측치 처리

In [10]:
def binning_age(x):
    if np.isnan(x):
        return np.nan
    elif x < 20:
        x = 10
    elif 20 <= x < 30:
        x = 20
    elif 30 <= x < 40:
        x = 30
    elif 40 <= x < 50:
        x = 40
    elif 50 <= x < 60:
        x = 60 
    elif x >= 60:
        x = 60
    return x

In [11]:
# 인덱싱 처리된 데이터 조인
train_df = df_train_ratings.merge(df_users, on='user_id', how='left').merge(df_books[['isbn', 'category', 'publisher', 'language', 'book_author', 'year_of_publication']], on='isbn', how='left')
test_df = df_test_ratings.merge(df_users, on='user_id', how='left').merge(df_books[['isbn', 'category', 'publisher', 'language', 'book_author', 'year_of_publication']], on='isbn', how='left')

In [12]:
# 1. 편차의 평균이 적은 column 두 개를 활용해 결측치 채우기
# 'book_author'와 'location_city'를 기준으로 train_df의 'age' 컬럼의 평균을 계산
age_means = train_df.groupby(["book_author", "location_city"])['age'].mean()

# train_df의 'age' 컬럼에서 결측치를 가진 행의 인덱스를 가져오기
null_age_idx = train_df[train_df['age'].isnull()].index

# 결측치를 채울 값들을 저장할 리스트
age_fill_values = []

for idx in tqdm(null_age_idx):
    author = train_df.loc[idx, 'book_author']
    city = train_df.loc[idx, 'location_city']
    # 'book_author'와 'location_city'가 일치하는 데이터의 평균값으로 결측치를 채우기.
    fill_value = age_means[author, city]
    age_fill_values.append(fill_value)

# 결측치를 채운 값들로 'age' 컬럼을 업데이트.
train_df.loc[null_age_idx, 'age'] = age_fill_values

100%|██████████| 92662/92662 [00:04<00:00, 22383.78it/s]


In [13]:
# 2. 도시의 평균으로 결측치 채우기
# 'location_city'를 기준으로 train_df의 'age' 컬럼의 평균을 계산
age_means_by_city = train_df.groupby('location_city')['age'].mean()

# 'age' 컬럼에서 결측치를 가진 행들의 'location_city' 값을 가져옵니다.
null_age_cities = train_df.loc[train_df['age'].isnull(), 'location_city']

# 'location_city'에 대한 평균값으로 결측치를 채웁니다.
train_df['age'].fillna(value=null_age_cities.map(age_means_by_city), inplace=True)

In [14]:
# 3. 전체 평균으로 결측치 채우기
train_df['age'] = train_df['age'].fillna(train_df['age'].mean())

In [17]:
train_df['binning_age'] = train_df['age'].apply(binning_age)
train_df['binning_age'] = train_df['binning_age'].astype('str')

In [18]:
train_df = train_df.drop('age', axis=1)

## 2) Books

In [19]:
for column in train_df.columns:
    if column == 'isbn':
        continue
    if train_df[column].dtype == object:
        # print(column)
        train_df[column] = train_df[column].apply(str_lower)

### 2-1) Publisher Number(pnumber) feature 새로 생성 : isbn 활용

In [20]:
train_df['pnumber'] = train_df['isbn'].apply(lambda x : x[:3])

### 2-2) category 결측치 해결

In [21]:
# book author 이용해서 해결 (같은 작가가 쓴 카테고리는 동일할 것으로 예상)
train_df.loc[train_df[train_df['category'].notnull()].index, 'category'] = train_df[train_df['category'].notnull()]['category'].apply(lambda x: re.sub('[\W_]+',' ',x).lower().strip())

temp = train_df.groupby('book_author')['category'].value_counts().groupby('book_author').idxmax().apply(lambda x : x[-1])
temp = defaultdict(lambda : np.nan, temp.to_dict())

feature2index = dict(zip(train_df.columns, range(len(train_df.columns))))
train_df['category'] = train_df.apply(lambda x : temp[x[feature2index['book_author']]] if type(x[feature2index['category']]) == float and np.isnan(x[feature2index['category']]) else x[feature2index['category']], axis = 1)

In [22]:
# pnumber 이용해서 해결 (같은 출판사에서 나온 카테고리는 동일할 것으로 예상)
temp = train_df.groupby('pnumber')['category'].value_counts().groupby('pnumber').idxmax().apply(lambda x : x[-1])
temp = defaultdict(lambda : np.nan, temp.to_dict())

feature2index = dict(zip(train_df.columns, range(len(train_df.columns))))
train_df['category'] = train_df.apply(lambda x : temp[x[feature2index['pnumber']]] if type(x[feature2index['category']]) == float and np.isnan(x[feature2index['category']]) else x[feature2index['category']], axis = 1)

In [24]:
# category가 한 단어로 기재된 경우로 통일
words = defaultdict(int)
for value in train_df['category'].values:
    try:
        if len(value.split()) == 1: # category가 한 단어로 기재된 경우
            words[value] += 1
    except:
        pass
categories = [(value, key) for key,value in words.items()] # category가 한 단어로 기재된 case마다 몇번 등장했는지
categories.sort(reverse = True)

In [26]:
for _, category in tqdm(categories):
    train_df.loc[train_df[train_df['category'].str.contains(category,na=False)].index,'category_high'] = category

100%|██████████| 1174/1174 [02:16<00:00,  8.62it/s]


### 2-3) language 결측치 해결

In [27]:
# 국가코드를 하나로 엮은 것에서 가장 많이 나온 MAXIMUM 언어로 결측치 해결
def country_code(isbn:str):
    prefix_1 = ('0','1','2','3','4','5','7')
    prefix_2 = tuple(map(str,range(80,94)))
    prefix_3 = tuple(list(map(str,range(950,960)))+list(map(str,range(961,969)))+list(map(str,range(970,985)))+['986','987'])
    if isbn.startswith(prefix_1):
        return isbn[0]
    elif isbn.startswith(prefix_2):
        return isbn[:2]
    elif isbn.startswith(prefix_3):
        return isbn[:3]
    else:
        return np.NaN
    
train_df['country_code'] = train_df['isbn'].map(country_code)
# 각 country에서 가장 많이 이용하는 language를 담은 dict 생성
top_languages = {}
for country_code, group in train_df.groupby('country_code'):
    try:
        top_language = group['language'].value_counts().idxmax()
        top_languages[country_code] = top_language
    except:
        pass
train_df['most_used_language'] = train_df['country_code'].map(top_languages)
# 해당 dict 이용해서 language 결측치 해결
train_df.loc[train_df[train_df['language'].isnull()].index,'language'] = train_df.loc[train_df[train_df['language'].isnull()].index,'most_used_language']
train_df = train_df.drop(['country_code','most_used_language'],axis=1)

In [28]:
# book author 이용해서 해결(같은 작가는 동일한 언어의 책을 낼 것으로 예상)
temp = train_df.groupby('book_author')['language'].value_counts().groupby('book_author').idxmax().apply(lambda x : x[-1])
temp = defaultdict(lambda : np.nan, temp.to_dict())

feature2index = dict(zip(train_df.columns, range(len(train_df.columns))))
train_df['language'] = train_df.apply(lambda x : temp[x[feature2index['book_author']]] if type(x[feature2index['book_author']]) == float and np.isnan(x[feature2index['language']]) else x[feature2index['language']], axis = 1)

In [30]:
# pnumber 이용해서 해결(같은 출판사는 동일한 언어의 책을 낼 것으로 예상)
temp = train_df.groupby('pnumber')['language'].value_counts().groupby('pnumber').idxmax().apply(lambda x : x[-1])
temp = defaultdict(lambda : np.nan, temp.to_dict())

feature2index = dict(zip(train_df.columns, range(len(train_df.columns))))
train_df['language'] = train_df.apply(lambda x : temp[x[feature2index['pnumber']]] if type(x[feature2index['language']]) == float and np.isnan(x[feature2index['language']]) else x[feature2index['language']], axis = 1)

### 2-4) year_of_publication binning

In [38]:
def binning_year(x):
    if x < 1970:
        return '1970'
    elif 1970 <= x < 1980:
        return '1980'
    elif 1980 <= x < 1990:
        return '1990'
    elif 1990 <= x < 2000:
        return '2000'
    else:
        return 'Early'
    
train_df['binning_year'] = train_df['year_of_publication'].apply(binning_year)

### 2-5) book_author cleaning

In [39]:
train_df['book_author'].fillna('unknown',inplace=True)
train_df['book_author'] = train_df['book_author'].apply(lambda x : re.sub(r'[^a-zA-Z0-9]', '', x))

In [41]:
train_df = train_df.drop(['year_of_publication','publisher','category'],axis=1)

In [42]:
for column in train_df.columns:
    train_df[column] = train_df[column].fillna('unknown')

In [44]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306795 entries, 0 to 306794
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           306795 non-null  int64 
 1   isbn              306795 non-null  object
 2   rating            306795 non-null  int64 
 3   location_city     306795 non-null  object
 4   location_state    306795 non-null  object
 5   location_country  306795 non-null  object
 6   language          306795 non-null  object
 7   book_author       306795 non-null  object
 8   binning_age       306795 non-null  object
 9   pnumber           306795 non-null  object
 10  category_high     306795 non-null  object
 11  binning_year      306795 non-null  object
dtypes: int64(2), object(10)
memory usage: 30.4+ MB
